In [1]:
#!pip install spotipy

In [2]:
import pandas as pd 
import spotipy 
import json
import datetime 
import os

sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 

In [3]:
# Credenciales
cid ="xxxxxxxxxx" 
secret = "xxxxxxxxxx" 

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

In [5]:
# Obtener la fecha de consulta para agregarla al nombre de los datos que descargaremos
fecha_actual = datetime.date.today()
anio = fecha_actual.isocalendar()[0]
semana = fecha_actual.isocalendar()[1]

In [7]:
file_output_top_50_global = "top_50_global_" + str(anio) + "_semana_" + str(semana) + ".txt"
file_output_top_50_global

'top_50_global_2022_semana_36.txt'

In [8]:
# Utilizar el id de la playlist de su preferencia
top_50_global = sp.user_playlist_tracks("leonardo.marintellez", "37i9dQZEVXbMDoHDwVN2tF")

In [9]:
# La consulta nos regresa un objeto en formato json
print(top_50_global.keys())

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])


In [10]:
# Inspeccionar los datos
print(json.dumps(top_50_global['items'], indent = 4))

[
    {
        "added_at": "2022-09-08T10:21:32Z",
        "added_by": {
            "external_urls": {
                "spotify": "https://open.spotify.com/user/"
            },
            "href": "https://api.spotify.com/v1/users/",
            "id": "",
            "type": "user",
            "uri": "spotify:user:"
        },
        "is_local": false,
        "primary_color": null,
        "track": {
            "album": {
                "album_type": "album",
                "artists": [
                    {
                        "external_urls": {
                            "spotify": "https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3"
                        },
                        "href": "https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3",
                        "id": "6KImCVD70vtIoJWnq6nGn3",
                        "name": "Harry Styles",
                        "type": "artist",
                        "uri": "spotify:artist:6KImCVD70vtIoJWnq6nGn3"
 

In [11]:
tracks = top_50_global['items']
len(tracks)

50

In [12]:
# Extraer el id de las canciones
ids = [] 
for i in range(len(tracks)): 
    ids.append(tracks[i]["track"]["id"]) 

In [13]:
# Obtener las características de las canciones
features=pd.DataFrame(sp.audio_features(ids))

In [14]:
features.shape

(50, 18)

In [15]:
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4Dvkj6JhhA12EX05fT7y2e,spotify:track:4Dvkj6JhhA12EX05fT7y2e,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...,167303,4
1,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,audio_features,2tTmW7RDtMQtBk7m2rYeSw,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,https://api.spotify.com/v1/tracks/2tTmW7RDtMQt...,https://api.spotify.com/v1/audio-analysis/2tTm...,198938,4
2,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
3,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,audio_features,1IHWl5LamUGEuP4ozKQSXZ,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4
4,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,audio_features,5ww2BF9slyYgNOk37BlC4u,spotify:track:5ww2BF9slyYgNOk37BlC4u,https://api.spotify.com/v1/tracks/5ww2BF9slyYg...,https://api.spotify.com/v1/audio-analysis/5ww2...,162638,4


In [16]:
#Extraer otros datos útiles: artista, canción,... 

num_top = [(i+1) for i in range(len(tracks))]

artist_name = []
song_name = []
popularity = []

for item in top_50_global['items']:
    artist_name.append(item['track']['album']['artists'][0]['name'])
    song_name.append(item['track']['name'])
    popularity.append(item['track']['popularity'])

In [17]:
# Agregar las características previas al dataframe
features['artist_name'] = artist_name
features['song_name'] = song_name
features['num_top'] = num_top
features['popularity'] = popularity

In [18]:
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_name,song_name,num_top,popularity
0,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,...,4Dvkj6JhhA12EX05fT7y2e,spotify:track:4Dvkj6JhhA12EX05fT7y2e,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...,167303,4,Harry Styles,As It Was,1,93
1,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,...,2tTmW7RDtMQtBk7m2rYeSw,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,https://api.spotify.com/v1/tracks/2tTmW7RDtMQt...,https://api.spotify.com/v1/audio-analysis/2tTm...,198938,4,Bizarrap,"Quevedo: Bzrp Music Sessions, Vol. 52",2,100
2,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,...,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4,Bad Bunny,Me Porto Bonito,3,98
3,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,...,1IHWl5LamUGEuP4ozKQSXZ,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4,Bad Bunny,Tití Me Preguntó,4,97
4,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,...,5ww2BF9slyYgNOk37BlC4u,spotify:track:5ww2BF9slyYgNOk37BlC4u,https://api.spotify.com/v1/tracks/5ww2BF9slyYg...,https://api.spotify.com/v1/audio-analysis/5ww2...,162638,4,Manuel Turizo,La Bachata,5,95


In [19]:
#os.getcwd()

In [20]:
features.to_csv(file_output_top_50_global, sep='|', index = False)